<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/01_analisis_de_valoraciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Valoraciones de Restaurantes

## Ejercicio Práctico de Expresiones Regulares y PLN
Este notebook presenta un ejercicio práctico para analizar reseñas de restaurantes utilizando expresiones regulares y procesamiento de lenguaje natural. El objetivo es extraer información valiosa de las valoraciones y realizar diversos análisis lingüísticos.

## 1. Configuración del Entorno

Primero, necesitamos instalar y configurar las dependencias necesarias:

In [1]:
!pip install spacy

!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 39.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Bibliotecas Utilizadas

-`spacy`: Para procesamiento de lenguaje natural    
-`pandas`: Para manipulación de datos     
-`json`: Para cargar datos JSON    
-`re`: Para expresiones regulares     
-`collections`: Para conteo de elementos

## 2. Carga y Exploración de Datos

Los datos consisten en reseñas de restaurantes con la siguiente estructura:

- `id`: Identificador único de la reseña    
- `rating`: Valoración (1-5 estrellas)    
- `text`: Texto de la reseña

In [4]:
# 1. Importaciones y Configuración Inicial
import json
import spacy
import pandas as pd
import re
from collections import Counter

# Cargar el modelo de español
nlp = spacy.load('es_core_news_sm')

# carga de Datos
with open('valoraciones_restaurante.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# crear DataFrame
df = pd.DataFrame(data['reviews']).set_index('id')

# Mostrar información básica
print("Forma del DataFrame:", df.shape)
print("\nDistribución de ratings:")
print(df['rating'].value_counts().sort_index())
df

Forma del DataFrame: (18, 2)

Distribución de ratings:
rating
1    2
2    3
3    3
4    5
5    5
Name: count, dtype: int64


,rating,text
id,,
1,5,Restaurante Il Gusto: El restaurante italiano ...
2,2,Bar La Esquina: La comida llegó fría y sin sab...
3,4,Restaurante Casa Valencia: La paella estaba bi...
4,3,Restaurante Sakura: El sushi es decente pero n...
5,1,Bar & Grill El Gaucho: Terrible experiencia. L...
6,5,Bar Los Caracoles: Las tapas son extraordinari...
7,4,Restaurante El Rincón: El menú del día ofrece ...
8,2,Pizzería Bella Napoli: La pizza llegó con ingr...
9,5,Restaurante Verde & Sano: El restaurante veget...


## 3. Análisis de Establecimientos

### 3.1 Extracción de Nombres

Utilizamos expresiones regulares para extraer el nombre completo del establecimiento:

In [5]:
def extraer_establecimientos(texto):
    """Extrae el nombre completo del establecimiento (tipo + nombre)"""
    patron = r'^(Restaurante|Bar|Cafetería|Pizzería|Taberna)\s([^:]+):'
    match = re.search(patron, texto)
    if match:
        return match.group(0)[:-1]  # Eliminamos los dos puntos del final
    return None

# Aplicar la extracción a todas las reseñas
df['establecimiento'] = df['text'].apply(extraer_establecimientos)

# Mostrar todos los establecimientos
print("=== Todos los establecimientos encontrados ===")
for establecimiento in df['establecimiento'].dropna():
    print(establecimiento)

=== Todos los establecimientos encontrados ===
Restaurante Il Gusto
Bar La Esquina
Restaurante Casa Valencia
Restaurante Sakura
Bar & Grill El Gaucho
Bar Los Caracoles
Restaurante El Rincón
Pizzería Bella Napoli
Restaurante Verde & Sano
Restaurante Gran Buffet
Cafetería Dulce & Salado
Bar La Tasca
Restaurante Fusión Asia
Restaurante El Antiguo
Bar & Burger El Garaje
Taberna La Bodeguita
Restaurante Mar Azul
Cafetería El Desayuno


## 3.2 Clasificación por Tipo

Extraemos y separamos el tipo y nombre del establecimiento:

In [6]:
def extraer_tipo_y_nombre(texto):
    """Extrae por separado el tipo de establecimiento y su nombre"""
    patron = r'^(Restaurante|Bar|Cafetería|Pizzería|Taberna)\s([^:]+):'
    match = re.search(patron, texto)
    if match:
        return {
            'tipo': match.group(1),
            'nombre': match.group(2)
        }
    return {'tipo': None, 'nombre': None}

# Aplicar la extracción detallada
resultados = df['text'].apply(extraer_tipo_y_nombre)
df['tipo'] = resultados.apply(lambda x: x['tipo'])
df['nombre'] = resultados.apply(lambda x: x['nombre'])

# 4. Análisis por tipo de establecimiento
print("\n=== Establecimientos por tipo ===")
for tipo in df['tipo'].unique():
    if tipo is not None:
        print(f"\nEstablecimientos de tipo '{tipo}' encontrados:")
        establecimientos = df[df['tipo'] == tipo]
        for _, row in establecimientos.iterrows():
            print(f"- {row['nombre']} (Rating: {row['rating']}⭐)")


=== Establecimientos por tipo ===

Establecimientos de tipo 'Restaurante' encontrados:
- Il Gusto (Rating: 5⭐)
- Casa Valencia (Rating: 4⭐)
- Sakura (Rating: 3⭐)
- El Rincón (Rating: 4⭐)
- Verde & Sano (Rating: 5⭐)
- Gran Buffet (Rating: 3⭐)
- Fusión Asia (Rating: 5⭐)
- El Antiguo (Rating: 2⭐)
- Mar Azul (Rating: 3⭐)

Establecimientos de tipo 'Bar' encontrados:
- La Esquina (Rating: 2⭐)
- & Grill El Gaucho (Rating: 1⭐)
- Los Caracoles (Rating: 5⭐)
- La Tasca (Rating: 1⭐)
- & Burger El Garaje (Rating: 4⭐)

Establecimientos de tipo 'Pizzería' encontrados:
- Bella Napoli (Rating: 2⭐)

Establecimientos de tipo 'Cafetería' encontrados:
- Dulce & Salado (Rating: 4⭐)
- El Desayuno (Rating: 4⭐)

Establecimientos de tipo 'Taberna' encontrados:
- La Bodeguita (Rating: 5⭐)


### 3.3 Estadísticas por tipo de Establecimiento

In [7]:
print("Número total de establecimientos:", len(df['establecimiento'].dropna()))
print("\nNúmero de establecimientos por tipo:")
print(df['tipo'].value_counts().to_string())

# 6. Rating promedio por tipo de establecimiento
print("\nRating promedio por tipo de establecimiento:")
print(df.groupby('tipo')['rating'].mean().round(2).to_string())

Número total de establecimientos: 18

Número de establecimientos por tipo:
tipo
Restaurante    9
Bar            5
Cafetería      2
Pizzería       1
Taberna        1

Rating promedio por tipo de establecimiento:
tipo
Bar            2.60
Cafetería      4.00
Pizzería       2.00
Restaurante    3.78
Taberna        5.00


## 4. Análisis Lingüistico

### 4.1 Análisis de Reseñas

Utilizamos spaCy para realizar un análisis lingüístico de las reseñas:

In [8]:
def analizar_review(texto):
    """
    Analiza una reseña y extrae información básica usando spaCy
    """
    # Procesar el texto con spaCy
    doc = nlp(texto)

    # Extraer elementos básicos
    analisis = {
        'adjetivos': [token.text for token in doc if token.pos_ == 'ADJ'],
        'sustantivos': [token.text for token in doc if token.pos_ == 'NOUN'],
        'verbos': [token.text for token in doc if token.pos_ == 'VERB']
    }

    return analisis

# 3. Analizar una reseña de ejemplo (la primera)
print("Ejemplo de análisis para una reseña:")
print("\nTexto original:")
print(df['text'].iloc[0])

resultado = analizar_review(df['text'].iloc[0])
print("\nElementos encontrados:")
print("Adjetivos:", resultado['adjetivos'])
print("Sustantivos:", resultado['sustantivos'])
print("Verbos:", resultado['verbos'])

Ejemplo de análisis para una reseña:

Texto original:
Restaurante Il Gusto: El restaurante italiano ofrece una pasta deliciosa y fresca. El chef prepara hábilmente los platos tradicionales. Los precios son bastante razonables para la calidad que ofrecen. El servicio fue impecable y el ambiente muy acogedor.

Elementos encontrados:
Adjetivos: ['italiano', 'deliciosa', 'fresca', 'tradicionales', 'razonables', 'impecable', 'acogedor']
Sustantivos: ['restaurante', 'pasta', 'chef', 'hábilmente', 'platos', 'precios', 'calidad', 'servicio', 'ambiente']
Verbos: ['ofrece', 'prepara', 'ofrecen']


### 4.2 Análisis de Frecuencuas

Analizamos las palabras más frecuentes por tipo y rating:

In [9]:
def palabras_frecuentes(df, rating, tipo_palabra):
    """
    Encuentra las palabras más comunes de un tipo específico para un rating dado
    """
    # Filtrar reseñas por rating
    reseñas_rating = df[df['rating'] == rating]['text']

    # Almacenar todas las palabras del tipo especificado
    palabras = []

    for texto in reseñas_rating:
        doc = nlp(texto)
        palabras.extend([token.text.lower() for token in doc if token.pos_ == tipo_palabra])

    # Contar frecuencias
    return Counter(palabras).most_common(5)


print("\nAnálisis por ratings:")
for rating in sorted(df['rating'].unique()):
    print(f"\nRating {rating} estrellas:")
    print("Adjetivos más comunes:", palabras_frecuentes(df, rating, 'ADJ'))
    print("Sustantivos más comunes:", palabras_frecuentes(df, rating, 'NOUN'))


Análisis por ratings:

Rating 1 estrellas:
Adjetivos más comunes: [('terrible', 1), ('quemada', 1), ('dura', 1), ('pésima', 1), ('despectiva', 1)]
Sustantivos más comunes: [('experiencia', 2), ('carne', 1), ('suela', 1), ('camarero', 1), ('quejas', 1)]

Rating 2 estrellas:
Adjetivos más comunes: [('malhumorado', 1), ('atento', 1), ('altos', 1), ('urgente', 1), ('diferentes', 1)]
Sustantivos más comunes: [('precios', 2), ('calidad', 2), ('comida', 1), ('fría', 1), ('sabor', 1)]

Rating 3 estrellas:
Adjetivos más comunes: [('decente', 1), ('extraordinario', 1), ('limpio', 1), ('decorado', 1), ('normales', 1)]
Sustantivos más comunes: [('local', 3), ('precios', 2), ('servicio', 2), ('sushi', 1), ('tipo', 1)]

Rating 4 estrellas:
Adjetivos más comunes: [('espectaculares', 2), ('primera', 2), ('preparada', 1), ('frescos', 1), ('amable', 1)]
Sustantivos más comunes: [('personal', 3), ('punto', 2), ('servicio', 2), ('día', 2), ('local', 2)]

Rating 5 estrellas:
Adjetivos más comunes: [('impr

## 5. Análisis Estadístico

### 5.1 Promedio de Palabras por Tipo

Calculamos estadísticas sobre el uso de diferentes tipos de palabras:

In [12]:
def promedio_palabras_por_tipo(texto):
    """
    Calcula el promedio de palabras por tipo en una reseña

    Args:
        texto (str): Texto de la reseña

    Returns:
        dict: Diccionario con el conteo de cada tipo de palabra
    """
    doc = nlp(texto)
    conteo = {'ADJ': 0, 'NOUN': 0, 'VERB': 0}
    for token in doc:
        if token.pos_ in conteo:
            conteo[token.pos_] += 1
    return conteo

valoracion = df['text'].iloc[0]
conteo = promedio_palabras_por_tipo(valoracion)
print(valoracion)
print(conteo)

Restaurante Il Gusto: El restaurante italiano ofrece una pasta deliciosa y fresca. El chef prepara hábilmente los platos tradicionales. Los precios son bastante razonables para la calidad que ofrecen. El servicio fue impecable y el ambiente muy acogedor.
{'ADJ': 7, 'NOUN': 9, 'VERB': 3}


## 6. Ejercicios propuestos para los estudiantes:
"""
1. ¿Qué adjetivos son más comunes en las reseñas positivas (4-5 estrellas)?
2. ¿Qué adjetivos son más comunes en las reseñas negativas (1-2 estrellas)?
3. ¿Cuáles son los sustantivos más mencionados en todas las reseñas?
4. ¿Cuántas palabras de cada tipo (ADJ, NOUN, VERB) hay en promedio en las reseñas?
"""